In [13]:
import pandas as pd
import glob
import os
import re

In [21]:
!pip install ekonlpy

     ---------------------------------------- 0.0/33.2 MB ? eta -:--:--
     -- ------------------------------------- 2.1/33.2 MB 9.0 MB/s eta 0:00:04
     --------------- ----------------------- 12.8/33.2 MB 31.0 MB/s eta 0:00:01
     --------------------------- ----------- 23.1/33.2 MB 36.5 MB/s eta 0:00:01
     --------------------------------------  32.8/33.2 MB 38.5 MB/s eta 0:00:01
     ---------------------------------------- 33.2/33.2 MB 36.9 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
   -------------------------------------- - 10.0/10.3 MB 47.8 MB/s eta 0:00:01
   ---------------------------------------- 10.3/10.

  You can safely remove it manually.
  You can safely remove it manually.


In [18]:
import re

def clean_news(text):
    if not isinstance(text, str):
        return ''
    
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    
    # 기자 이메일, 언론사 이름 제거
    text = re.sub(r'\[.+?기자\]|\[.+?뉴스\]|\[.+?제공\]', '', text)
    text = re.sub(r'\/사진제공?=.+?$', '', text)
    text = re.sub(r'\/사진?=.+?$', '', text)
    
    # 홍보 문구 제거
    text = re.sub(r'이\s*기사는\s*.+?\s*에\s*게재된\s*기사입니다\.', '', text)

    # 줄바꿈 공백으로
    text = text.replace('\u2028', ' ').replace('\u2029', ' ').replace('\n', ' ')
    
    # 한글, 영어, 숫자, 공백, 마침표만 남기기 (마침표는 문장 구분을 위해 남겨야 함)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s\.]', ' ', text)
    
    # 연속된 공백 하나로 줄이기
    text = ' '.join(text.split())
    
    return text

In [20]:
# 여러 언론사 클렌징 데이터 합치기
from tqdm import tqdm
tqdm.pandas()

folder_path = '../../db/news_contents/news_contents_*.csv'
file_list = glob.glob(folder_path)

all_data = []

for file in file_list:
    filename = os.path.basename(file)
    source_name = filename.split('_')[-1].replace('.csv', '')

    df = pd.read_csv(file, usecols=['date', 'content'])

    # 오염 데이터 제거용
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date'])

    temp_df = pd.DataFrame()
    temp_df['date'] = pd.to_datetime(df['date'])
    temp_df['date'] = temp_df['date'].dt.date

    temp_df['content'] = df['content'].progress_apply(clean_news)
    temp_df['tokens'] = ''
    temp_df['category'] = '뉴스'
    temp_df['source'] = source_name

    all_data.append(temp_df)
    print(f"✔ {source_name} 처리 완료 ({len(temp_df)}건)")

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    final_df = final_df[['date', 'content', 'tokens', 'category', 'source']]
    final_df.to_csv('news_preprocessed_integrated.csv', index=False, encoding='utf-8-sig')
    print(f"\n✨ 총 {len(final_df)}건의 '뉴스' 데이터 저장 완료")


100%|██████████| 71236/71236 [00:03<00:00, 21872.43it/s]


✔ 매일경제 처리 완료 (71236건)


100%|██████████| 89012/89012 [00:05<00:00, 17015.67it/s]


✔ 머니투데이 처리 완료 (89012건)


100%|██████████| 79208/79208 [00:04<00:00, 19770.41it/s]


✔ 한국경제 처리 완료 (79208건)

✨ 총 239456건의 '뉴스' 데이터 저장 완료


In [ ]:
from ekonlpy.sentiment import MPCK

input_file = 'news_preprocessed_integrated.csv'
output_file = 'news_preprocessed_output.csv'
batch_size = 5000 

if os.path.exists(output_file):
    print(f"♻️ 기존에 작업하던 '{output_file}'을 발견했습니다. 이어서 시작합니다.")
    df = pd.read_csv(output_file)
else:
    print(f"🆕 처음부터 시작합니다. '{input_file}'을 불러옵니다.")
    df = pd.read_csv(input_file)
    if 'tokens' not in df.columns:
        df['tokens'] = ""

mpck = MPCK()

def get_tokens(text):
    try:
        if not text or len(str(text)) < 10:
            return ""
        tokens = mpck.tokenize(text)
        ngrams = mpck.ngramize(tokens)
        return ", ".join(tokens + ngrams)
    except:
        return ""

to_process = df[df['tokens'].isna() | (df['tokens'] == "")].index
print(f"🚀 남은 작업량: {len(to_process)}건 / 전체: {len(df)}건")

try:
    for i, idx in enumerate(tqdm(to_process)):
        df.at[idx, 'tokens'] = get_tokens(df.at[idx, 'content'])
        
        if (i + 1) % batch_size == 0 or (i + 1) == len(to_process):
            df.to_csv(output_file, index=False, encoding='utf-8-sig')
            
except KeyboardInterrupt:
    print("\n🛑 사용자가 작업을 중단했습니다. 현재까지의 내용은 안전하게 저장되었습니다.")

print(f"✨ 모든 작업 완료! 결과 파일: {output_file}")

🆕 처음부터 시작합니다. 'news_preprocessed_integrated.csv'을 불러옵니다.
🚀 남은 작업량: 239456건 / 전체: 239456건


밤샘 토큰화 작업 중:   0%|          | 0/239456 [00:00<?, ?it/s]C:\Users\hg432\AppData\Local\Temp\ipykernel_26596\483549623.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '중구/NNG, 본점/NNG, 딜링룸/NNG, 현황/NNG, 판/NNG, 코스피/NNG, 표시/NNG, 되/VV, 연속/NNG, 상승/NNG, 하/VV, 코스피/NNG, 급락/NNG, 하/XSV, 선/NNG, 간신히/MAG, 지켰/VV, 간밤/NNG, 폐쇄/NNG, 해제/NNG, 불구/NNG, 금리/NNG, 인하/NNG, 기대/NNG, 약화/NNG, 평가/NNG, 논란/NNG, 불거지/VV, 매도/NNG, 심화/NNG, 하/XSV, 급락/NNG, 영향/NNG, 풀이/NNG, 되/VV, 코스피/NNG, 대비/NNG, 포인트/NNG, 내리/VV, 거래/NNG, 마치/VV, 지수/NNG, 대비/NNG, 포인트/NNG, 급락/NNG, 출발/NNG, 뒤/NNG, 낙폭/NNG, 확대/NNG, 하/VV, 간밤/NNG, 산업/NNG, 고평가/NNG, 우려/NNG, 미/NNG, fed/NNG, fed/NNG, fed/NNG, 매파/NNG, 통화/NNG, 긴축/NNG, 선호/NNG, 따르/VV, 금리동결/NNG, 전망/NNG, 급락/NNG, 영향/NNG, 분석/NNG, 되/VV, 지수/NNG, 대비/NNG, 포인트/NNG, 하락/NNG, 나스닥/NNG, 포인트/NNG, 내리/VV, 장/NNG, 마감/NNG, 하/VV, 다우존스산업평균지수/NNG, 대비/NNG, 포인트/NNG, 하락/NNG, 거래/NNG, 마치/VV, 또/MAG, 티어/NNG, 주가/NNG, 급락/NNG, 하/VV, 하락/NNG, 하/VV, 금리/NNG, 인하/NNG, 기대

✨ 모든 작업 완료! 결과 파일: news_preprocessed_output.csv


In [ ]:
def paper_based_ngramize(tokens, max_n=5):
    # 논문 4대 품사 + 사용자님의 '부정어(VX)' 추가
    keep_tags = ['NNG', 'VV', 'VA', 'MAG', 'VX']
    
    # 1. 품사 필터링
    filtered = [w for w in tokens if w.split('/')[1] in keep_tags]
    
    ngram_results = []
    # 2. 1-gram부터 5-gram까지 생성 (논문 방식)
    for pos in range(len(filtered)):
        for n in range(1, max_n + 1):
            if pos + n <= len(filtered):
                ngram = ";".join(filtered[pos : pos + n])
                ngram_results.append(ngram)
                
    return ngram_results